In [82]:
import pandas as pd

data = pd.read_csv(r'C:\Users\Lenovo\Desktop\AgriCast Predictive Analytics for Crop Production\Data\Scaled\crop_production_scaled.csv').iloc[:10000, :]

In [83]:
data.head()

,Crop_Year,Crop,Area,Production,Season_Autumn,Season_Kharif,Season_Rabi,Season_Summer,Season_Whole Year,Season_Winter
0,2000,1163.044069,1.195849,2000.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2000,300.938895,-0.727498,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2000,1538.321946,-0.573876,321.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2000,3066.720805,-0.460196,641.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2000,288.924697,0.375507,165.0,0.0,0.0,0.0,0.0,1.0,0.0


In [84]:
x = data.drop(columns=['Production'], axis=1)
y = data['Production']

In [85]:
x.head()

,Crop_Year,Crop,Area,Season_Autumn,Season_Kharif,Season_Rabi,Season_Summer,Season_Whole Year,Season_Winter
0,2000,1163.044069,1.195849,0.0,1.0,0.0,0.0,0.0,0.0
1,2000,300.938895,-0.727498,0.0,1.0,0.0,0.0,0.0,0.0
2,2000,1538.321946,-0.573876,0.0,1.0,0.0,0.0,0.0,0.0
3,2000,3066.720805,-0.460196,0.0,0.0,0.0,0.0,1.0,0.0
4,2000,288.924697,0.375507,0.0,0.0,0.0,0.0,1.0,0.0


In [74]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [75]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

In [76]:
import xgboost as xg

model = xg.XGBRegressor(
    objective='reg:linear', n_estimators=200, seed=42, max_depth=None
)

model.fit(xtrain, ytrain)

c:\Users\Lenovo\Desktop\AgriCast Predictive Analytics for Crop Production\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:45:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [77]:
ypred = model.predict(xtest)

In [78]:
from sklearn.metrics import r2_score

r2_score(ytest, ypred)

0.8232162841517061

# R2 Score: `0.82`

### GridSearchCV

In [45]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [None, 5, 7]
}

grid_search = GridSearchCV(xg.XGBRegressor(), param_grid, scoring='r2', cv=5, verbose=2, n_jobs=-1)

In [46]:
grid_search.fit(xtrain, ytrain)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [None, 5, 7],
                         'n_estimators': [100, 300, 500]},
             scoring='r2', verbose=2)

In [47]:
grid_search.best_params_

{'learning_rate': 0.05, 'max_depth': None, 'n_estimators': 300}

### True Model

In [68]:
from xgboost import XGBRegressor

model = XGBRegressor(
    learning_rate=0.05,
    max_depth = None,
    n_estimators= 300
)

model.fit(xtrain, ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [69]:
ypred = model.predict(xtest)

In [70]:
from sklearn.metrics import r2_score

r2_score(ytest, ypred)

0.8292967217631264

# R2 Score: 0.83